# Exemples d'explorateurs de fichiers

Ce notebook regroupe des exemple d'explorateurs de fichiers récoltés sur le web. Chaque exemple est précédé de l'URL où il a été trouvé.

## Source: https://gist.github.com/thomasaarholt/b89b3f29ad82cd32176f2605834fa8ac

In [2]:
import ipywidgets as widgets

import os
from pathlib import Path
cwd = Path(os.getcwd())

FOLDERLABEL = '-------FOLDERS-------'
FILESLABEL = '-------FILES-------'

def get_folder_contents(folder):
    'Gets contents of folder, sorting by folder then files, hiding hidden things'
    folder = Path(folder)
    folders = [item.name for item in folder.iterdir() if item.is_dir() and not item.name.startswith('.')]
    files = [item.name for item in folder.iterdir() if item.is_file() and not item.name.startswith('.')]
    return [FOLDERLABEL] + sorted(folders) + [FILESLABEL] + sorted(files)

def go_to_address(address):
    address = Path(address)
    if address.is_dir():
        address_field.value = str(address)
        select.unobserve(selecting, names='value')
        select.options = get_folder_contents(folder=address)
        select.observe(selecting, names='value')
        select.value = None

def newaddress(value):
    go_to_address(address_field.value)
        
def selecting(value):
    if value['new'] and value['new'] not in [FOLDERLABEL, FILESLABEL]:
        path = Path(address_field.value)
        newpath = path / value['new']
        if newpath.is_dir():
            go_to_address(newpath)
            
        elif newpath.is_file():
            #some other condition
            pass

def parent(value):
    new = Path(address_field.value).parent
    go_to_address(new)
    
def load(value):
    filepath = path / value['new']
    # load using your favourite python software!
    
address_field = widgets.Text(value=str(cwd))
up_button = widgets.Button(description='Up')
select = widgets.Select(options=get_folder_contents(cwd), rows=15, value=None)
load_button = widgets.Button(description='Load')
box = widgets.HBox([widgets.VBox([address_field, select]), widgets.VBox([up_button, load_button])])

select.observe(selecting, names='value')
up_button.on_click(parent)
load_button.observe(load)
address_field.on_submit(newaddress)

box

## Source: https://pastebin.com/niqUTVy6

In [3]:
import os

import ipywidgets as widgets


class FileBrowser(object):
    def __init__(self):
        self.path = os.getcwd()
        self._update_files()
        
    def _update_files(self):
        self.files = list()
        self.dirs = list()
        if(os.path.isdir(self.path)):
            for f in os.listdir(self.path):
                ff = os.path.join(self.path, f)
                if os.path.isdir(ff):
                    self.dirs.append(f)
                else:
                    self.files.append(f)
        
    def widget(self):
        box = widgets.VBox()
        self._update(box)
        return box
    
    def _update(self, box):
        
        def on_click(b):
            if b.description == '..':
                self.path = os.path.split(self.path)[0]
            else:
                self.path = os.path.join(self.path, b.description)
            self._update_files()
            self._update(box)
        
        buttons = []
        if self.files:
            button = widgets.Button(description='..', background_color='#d0d0ff')
            button.on_click(on_click)
            buttons.append(button)
        for f in self.dirs:
            button = widgets.Button(description=f, background_color='#d0d0ff')
            button.on_click(on_click)
            buttons.append(button)
        for f in self.files:
            button = widgets.Button(description=f)
            button.on_click(on_click)
            buttons.append(button)
        box.children = tuple([widgets.HTML("<h2>%s</h2>" % (self.path,))] + buttons)

# example usage:
#   f = FileBrowser()
#   f.widget()
#   <interact with widget, select a path>
# in a separate cell:
#   f.path # returns the selected path

In [4]:
f = FileBrowser()
f.widget()

## Source: https://stackoverflow.com/a/48451751/348867

In [5]:
import sys
import os
import ipywidgets as ui
from IPython.display import display

class PathSelector():

    def __init__(self,start_dir,select_file=True):
        self.file        = None 
        self.select_file = select_file
        self.cwd         = start_dir
        self.select      = ui.SelectMultiple(options=['init'],value=(),rows=10,description='') 
        self.accord      = ui.Accordion(children=[self.select]) 

        self.accord.selected_index = None # Start closed (showing path only)
        self.refresh(self.cwd)
        self.select.observe(self.on_update,'value')

    def on_update(self,change):
        if len(change['new']) > 0:
            self.refresh(change['new'][0])

    def refresh(self,item):
        path = os.path.abspath(os.path.join(self.cwd,item))

        if os.path.isfile(path):
            if self.select_file:
                self.accord.set_title(0,path)  
                self.file = path
                self.accord.selected_index = None
            else:
                self.select.value = ()

        else: # os.path.isdir(path)
            self.file = None 
            self.cwd  = path

            # Build list of files and dirs
            keys = ['[..]']; 
            for item in os.listdir(path):
                if item[0] == '.':
                    continue
                elif os.path.isdir(os.path.join(path,item)):
                    keys.append('['+item+']'); 
                else:
                    keys.append(item); 

            # Sort and create list of output values
            keys.sort(key=str.lower)
            vals = []
            for k in keys:
                if k[0] == '[':
                    vals.append(k[1:-1]) # strip off brackets
                else:
                    vals.append(k)

            # Update widget
            self.accord.set_title(0,path)  
            self.select.options = list(zip(keys,vals)) 
            with self.select.hold_trait_notifications():
                self.select.value = ()

f = PathSelector('.')
display(f.accord)

Accordion(children=(SelectMultiple(options=(('[..]', '..'), ('[data]', 'data'), ('ColorQC.ipynb', 'ColorQC.ipy…

In [8]:
f.file

'/home/al/backup-usb/formation/Python-Formation-Initiation-Fonctions.pdf'